In [1]:
import pandas as pd

In [2]:
df_perf = pd.read_csv('data/aggr_data.csv')
df_scrape = pd.read_csv('data/full_scraped.csv')

df_perf.drop(['old_index','publish_date_equal_to_date','version_id'], axis=1, inplace=True)
df_perf = df_perf[['page_version_id','page_id','version_id_new','date', 'publish_date', 'word_count', 'url', 'page_name','title',
       'classification_product', 'classification_type', 'authors','daily_likes',
       'daily_dislikes', 'video_play', 'page_impressions', 'clickouts',
       'external_clicks', 'external_impressions']]

FileNotFoundError: [Errno 2] No such file or directory: 'data/aggr_data.csv'

### Target Variables:
Impressions, Clicks, CTR (click-through-rate). The latter we create in the next step:

In [ ]:
# Calculate the Click through rate based on external clicks and impressions
df_perf['ctr'] = df_perf['external_clicks'] / df_perf['external_impressions'] *100

### Transformation and feature engineering of scraped data

Extract the last part of the URL to analyze it and inhibit duplicate data with classification_type

In [ ]:
# Function to extract last part of URL and clean it
def extract_last_part(url):
    url_text = url.rsplit('/', 1)[-1]
    cleaned_url = url_text.split('_')[0]
    cleaned_url_list = cleaned_url.split('-')
    return cleaned_url_list

# Apply the function to create a new column
df_scrape['url_text'] = df_scrape['url'].apply(extract_last_part)

# Sum up all list items per ongoing Version ID and merge with original df
df_feat = pd.merge(df_scrape, df_scrape.groupby('page_id')['url_text'].apply(lambda x: list(set(sum(x, [])))).reset_index(name='merged_url'), on='page_id', how='left')

In [ ]:
#Transform media column
def media_type(df, media_type):
    if 'img-wrapper' in media_type or any(item in media_type for item in ['image-gallery', 'mb-lg-7', 'mb-8']):
        return 'img'
    elif any(item in media_type for item in ['mb-3', 'video-player', 'recobar']):
        return 'video'
    else:
        return 'other'

df_feat['media_type'] = df_scrape['media_type'].apply(lambda x: media_type(df_feat, x))

In [ ]:
# Title length
df_feat['meta_title_len'] = df_feat['meta_title'].str.len()

# Meta description length
df_feat['meta_desc_len'] = df_feat['meta_description'].str.len()

# H1 length
df_feat['h1_len'] = df_feat['h1'].str.len()

# Abstract length
df_feat['abstract_len'] = df_feat['abstract'].str.len()

# URL length
df_feat['merged_url_len'] = df_feat['merged_url'].str.len()

In [ ]:
print(df_feat.columns)
print(df_perf.columns)

Index(['page_id', 'url', 'h1', 'author', 'date_scraped', 'abstract',
       'main_text_length', 'meta_title', 'meta_description', 'meta_image_url',
       'media_type', 'page_img_size', 'url_text', 'merged_url',
       'meta_title_len', 'meta_desc_len', 'h1_len', 'abstract_len',
       'merged_url_len'],
      dtype='object')
Index(['page_version_id', 'page_id', 'version_id_new', 'date', 'publish_date',
       'word_count', 'url', 'page_name', 'title', 'classification_product',
       'classification_type', 'authors', 'daily_likes', 'daily_dislikes',
       'video_play', 'page_impressions', 'clickouts', 'external_clicks',
       'external_impressions', 'ctr'],
      dtype='object')


### Merging scraped and provided data

In [ ]:
merge_keys = ['page_id', 'url']
df_full = pd.merge(left=df_perf,right=df_feat,how='left',on=merge_keys)

In [ ]:
df_full.columns

Index(['page_version_id', 'page_id', 'version_id_new', 'date', 'publish_date',
       'word_count', 'url', 'page_name', 'title', 'classification_product',
       'classification_type', 'authors', 'daily_likes', 'daily_dislikes',
       'video_play', 'page_impressions', 'clickouts', 'external_clicks',
       'external_impressions', 'ctr', 'h1', 'author', 'date_scraped',
       'abstract', 'main_text_length', 'meta_title', 'meta_description',
       'meta_image_url', 'media_type', 'page_img_size', 'url_text',
       'merged_url', 'meta_title_len', 'meta_desc_len', 'h1_len',
       'abstract_len', 'merged_url_len'],
      dtype='object')

<span style='color:red'>to be done
### Finetune versions

In [ ]:
col_perf = ['page_id', 'page_version_id','date','external_clicks', 'external_impressions','video_play', 'page_impressions', 'clickouts','daily_likes', 'daily_dislikes','ctr']

col_gen = ['page_version_id', 'page_id', 'version_id_new', 'publish_date', 'word_count', 'url', 'page_name', 'title', 'classification_product', 'classification_type', 'authors']

# Define aggregation rules for aggregation by date
agg_to_date = {
    'page_version_id': 'sum', # Number of versions
    'page_id': 'first', # Id for matching
    #'date': 'first', # Prevent error
    'external_impressions': 'first',  # Metric is duplicated
    'external_clicks': 'first',   # Metric is duplicated
    'video_play': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'page_impressions': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'clickouts': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'daily_likes': 'first',   # Metric is duplicated
    'daily_dislikes': 'first',   # Metric is duplicated
    'ctr': 'first' # Metric is duplicated
}

# Aggregate data on a date level
df_perf_date = df_perf[col_perf].groupby(['page_id','date'],as_index=False).agg(agg_to_date)

In [ ]:
df_perf_date
#df_perf_date.shape

In [ ]:
# Define aggregation rules for aggregation by page_id
agg_to_page = {
    'page_version_id': 'first', # Id for matching
    'page_id': 'first',
    'date': 'first', # Prevent error
    'external_impressions': 'sum',  # Sum of all days
    'external_clicks': 'sum',   # Sum of all days
    'video_play': 'sum',   # Sum of all days
    'page_impressions': 'sum',   # Sum of all days
    'clickouts': 'sum',   # Sum of all days
    'daily_likes': 'sum',   # Sum of all days
    'daily_dislikes': 'sum',   # Sum of all days
    'ctr': 'median' # Meidan of all days # Could be recalculated alternatively
}

# Aggregate on a page_id level
df_perf_page = df_perf_date[col_perf].groupby(['page_id']).agg(agg_to_page)

print(df_perf_date.shape)

In [ ]:
df_gen_page = df_perf[col_gen].groupby(['page_id']).first()

df_agg = pd.merge(left=df_perf_date, right=df_gen_page, on='page_id',how='left')

# df_agg = df_agg[['date','publish_date', 'word_count', 'url', 'page_name', 'title',
#        'classification_product', 'classification_type', 'authors', 
#        'page_id','version_id_new', 'video_play', 'page_impressions', 'clickouts', 'daily_likes',
#        'daily_dislikes', 'ctr',  'external_impressions', 'external_clicks']]

print(df_gen_page.shape)
print(df_agg.shape)

In [ ]:
# # Create ongoing version_id that is unique for each version page_id combination
# df_perf['version_id_ong'] = df_perf['page_id'].astype(str) + '_' + df_perf['version_id_new'].astype(str)
# df_perf[['version_id_ong', 'page_id', 'version_id_new']].head()

Thomas:
We have three groups which depend on different concatenated unique keys:

#### Group 1: external_impressions and external_clicks:

page_id
date

#### Group 2: video_play, page_impressions, clickouts:

page_id
date
URL
Author (edge case)

#### Group 3: daily_likes, daily_dislikes:

page_id
date
publishe_date

In [ ]:
col_agg_1 = ['external_clicks', 'external_impressions']
col_agg_2 = ['video_play', 'page_impressions', 'clickouts']
col_agg_3 = ['daily_likes', 'daily_dislikes']

# Columns that don't need to be aggregated but are the same for each version
all_columns = df_perf.columns.tolist()
col = [c for c in all_columns if c not in col_agg_1 and c not in col_agg_2 and c not in col_agg_3]
# this includes: ['old_index', 'page_id', 'date', 'url', 'version_id_new', 'publish_date', 'word_count', 'words_scraped', 'classification_product', 'classification_type', 'page_name', 'authors', 'author_scraped', 'title', 'h1', 'abstract', 'last_update', 'image_url', 'version_id_ong']
col

In [ ]:
pd.set_option('display.max_colwidth', 20)

# Columns that differ on a daily basis and need to be aggregated with a certain rule
col_special = ['page_version_id','date','external_clicks', 'external_impressions','video_play', 'page_impressions', 'clickouts','daily_likes', 'daily_dislikes']

# Columns that don't need to be aggregated but are the same for each version
all_columns = df_perf.columns.tolist()
col = [c for c in all_columns if c not in col_special] + ['page_version_id'] + ['date']
# this includes: ['old_index', 'page_id', 'date', 'url', 'version_id_new', 'publish_date', 'word_count', 'words_scraped', 'classification_product', 'classification_type', 'page_name', 'authors', 'author_scraped', 'title', 'h1', 'abstract', 'last_update', 'image_url', 'version_id_ong']

# Aggregate by version for columns with simple duplicates
df_agg_col = df_perf[col].groupby('page_version_id').first()

# Aggregate by version for columns which need to be aggregated with a certain rule
agg_funcs = {
    'external_impressions': 'first',  # Metric is duplicated
    'external_clicks': 'first',   # Metric is duplicated
    'video_play': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'page_impressions': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'clickouts': 'sum',   # Metric sometimes differs highly for different URLs & authors
    'daily_likes': 'first',   # Metric is duplicated
    'daily_dislikes': 'first'   # Metric is duplicated
}

# Group by and apply aggregation functions
df_agg_special = df_perf[col_special].groupby(['date']).sum()
df_agg_special = df_perf[col_special].groupby(['page_version_id']).agg(agg_funcs)

df_agg = pd.merge(left=df_agg_col, right=df_agg_special, on='page_version_id',how='left')

df_agg = df_agg[['date','publish_date', 'word_count', 'url', 'page_name', 'title',
       'classification_product', 'classification_type', 'authors', 
       'page_id','version_id_new', 'video_play', 'page_impressions', 'clickouts', 'daily_likes',
       'daily_dislikes', 'ctr',  'external_impressions', 'external_clicks']]

print(df_agg_special.shape)
print(df_agg_col.shape)
print(df_agg.shape)

In [ ]:
df_agg